Каждый раз, обучая нейронку, мы сначала рандомно инициализируем веса, а после в ходе бэкпропа обучаем модель. Если мы сразу же угадываем хорошие веса, модель сходится быстрее. Иногда можно брать в качестве инициализации веса, полученные другими исследователями и на их основе дообучать модель под свой выход. Это здорово упрощает задачу обучения и экономит недели работы.

Transfer learning — это когда вы берёте чужую модель и адаптируете её под свою задачу. В этой тетрадке мы посмотрим на то, как в PyTorch можно это сделать.

В прошлый раз мы обсуждали историческое развитие разных нейросетевых архитектур от AlexNet (2012 года) до ResNet (2015 года). Сегодня мы возьмём предобученный ResNet-18 из Torchvision и переделаем его так, чтобы он начал решать задачу классификации изображений на новом датасете.

In [ ]:
import io
import requests
from pathlib import Path

import PIL
import numpy as np
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

# 1. Данные

Сегодня мы попробуем решить проблему тысячелетия вслед за [лучшими китайскими учёными](https://www.youtube.com/watch?v=vIci3C4JkL0).

![](https://www.semantics3.com/blog/content/images/downloaded_images/hot-dog-and-a-not-hot-dog-the-distinction-matters-code-included-8550067fb16/1-VrpXE1hE4rO1roK0laOd7g.png)

Мы будем отличать хот-доги от всего остального.

## 1.1. Скачиваем датасет

Данные мы возьмём из [соревнования на Kaggle](https://www.kaggle.com/c/hotdogornot). Поскольку в Colab затруднительно скачать данные напрямую с Kaggle (для этого нужно добывать API-ключ), я залил обучающую выборку из этого соревнования на [Google Drive](https://drive.google.com/file/d/1IkDqUUidWfB0l_OnO239OZMVCUONiJUF/view?usp=sharing). Скачать её оттуда можно при помощи команды `gdown`.

In [ ]:
data_root = Path('train_kaggle')

In [ ]:
if not data_root.exists():
    !gdown https://drive.google.com/uc?id=1IkDqUUidWfB0l_OnO239OZMVCUONiJUF
    !unzip -q kaggle_hotdogornot_train.zip
    assert data_root.exists()

## 1.2. Смотрим на датасет глазами

В Colab есть примитивный браузер файлов, который позволяет посмотреть, что мы там такое скачали. Кроме того, мы можем повыполнять разные команды, чтобы получить представление о содержимом датасета.

Консольные команды из ноутбука можно выполнять так:

In [ ]:
!echo 'this is a command'

In [ ]:
# Сколько файлов лежит в data_root?
<YOUR CODE>

In [ ]:
# Как называются первые несколько файлов?
<YOUR CODE>

In [ ]:
# Верно ли, что все файлы имеют имена вида <класс>_<число>.jpg?
import pandas as pd

records = []
for p in data_root.iterdir():
    # p.suffix — расширение файла
    # p.stem — имя без расширения
    
    <YOUR CODE>
    
    records.append({
        'name': name,
        'klass': klass,
        'num': num,
    })

df_files = pd.DataFrame(records)
df_files

In [ ]:
# Какие классы есть в датасете?
df_files.klass.unique()

In [ ]:
# Как выглядят случайные примеры?
PIL.Image.open(data_root / df_files[df_files.klass == 'hotdog'].sample().name.iloc[0])

## 1.3. Пишем обёртку `torch.utils.data.Dataset`

А теперь напишем обёртку-наследника `torch.utils.data.Dataset`, чтобы дальше работать с этим датасетом. Нам понадобится следующее:

* Распарсить все имена файлов и для каждого файла извлечь класс
* Захардкодить или иным способом зафиксировать порядок классов в датасете (чтобы он не менялся между обучением и использованием модели)
* Разделить датасет на обучающую и валидационную выборку со стратификацией

In [ ]:
from sklearn.model_selection import train_test_split

class HotDogOrNotDataset(torch.utils.data.Dataset):
    classes = ['pets', 'furniture', 'people', 'food', 'frankfurter', 'chili-dog', 'hotdog']
    
    def __init__(self, data_root: Path, split='train', transform=None):
        super().__init__()
        assert split in {'train', 'test'}, f'Unknown split value: {split}'
        self.data_root = data_root
        self.split = split
        self.transform = transform  # Преобразование, применяемое ко всем загружаемым элементам датасета
        
        self.class_to_idx = {klass: i for i, klass in enumerate(self.classes)}
        
        paths = sorted(data_root.iterdir(), key=lambda p: int(p.stem.split('_')[1]))
        indices_train, indices_test = train_test_split(
            range(len(paths)),
            stratify=[self.class_to_idx[self._path_to_class(p)] for p in paths],
            test_size=0.2, random_state=42)
        
        if split == 'train':
            indices = set(indices_train)
        else:
            indices = set(indices_test)
        
        self.filenames = [p.name for i, p in enumerate(paths) if i in indices]
        
    @staticmethod
    def _path_to_class(path: Path):
        """Given a path like {dataset_root}/{class}_{idx}.jpg, return class."""
        return path.stem.split('_')[0]
        
    def __getitem__(self, idx):
        path = self.data_root / self.filenames[idx]
        
        X = PIL.Image.open(path)
        y = self.class_to_idx[self._path_to_class(path)]
        
        # Применяем преобразование, заданное при инициализации. Именно так работают аугментации.
        if self.transform is not None:
            X = self.transform(X)
        
        return X, y
    
    def __len__(self):
        return len(self.filenames)
    
    def __repr__(self):
        return '\n'.join([
            'Dataset HotDogOrNot',
            f'    Number of datapoints: {len(self)}',
            f'    Root location: {self.data_root}',
            f'    Split: {self.split}',
        ])

dataset_valid = HotDogOrNotDataset(data_root, split='test')
dataset_valid

Проверим, что наш класс ведёт себя ожидаемым образом:

In [ ]:
X, y = dataset_valid[0]
print(dataset_valid.classes[y])
X

## 1.4. Преобразования валидационного датасета

Теперь для примера попробуем реализовать какие-нибудь преобразования датасета. Поскольку это не самая сложная и важная задача, не будем акцентировать на этом большое внимание и реализуем только два преобразования:

1. Изменение размеров изображения с сохранением пропорций;
2. Вырезание куска из центра изображения.

Здесь нужно сделать важное замечание. Мы целенаправленно используем для работы с изображениями библиотеку PIL (Python Imaging Library), потому что мы повторяем за torchvision, но, разумеется, нас никто не заставляет это делать. Мы могли бы пользоваться, например, библиотекой OpenCV и методами наподобие `cv2.imread` для загрузки изображений, возвращающими Numpy array, а не какие-то специализированные объекты типа `PIL.Image`. Альтернативная библиотека для аугментаций `albumentations` пошла именно по этому пути.

In [ ]:
class Resize(nn.Module):
    def __init__(self, size: int):
        super().__init__()
        assert isinstance(size, int)
        self.size = size
    
    def forward(self, image: PIL.Image):
        # image is a PIL.Image, not a torch.Tensor, so we need to use PIL.Image methods
        img_w, img_h = image.size
        
        # compute new_w and new_h
        <YOUR CODE>
        
        return image.resize((new_w, new_h))

assert Resize(256)(PIL.Image.new('RGB', (719, 960))).size in {(256, 342), (256, 341)}
assert Resize(256)(PIL.Image.new('RGB', (960, 719))).size in {(342, 256), (341, 256)}

Resize(256)(dataset_valid[0][0])

In [ ]:
from typing import Union, Tuple

class CenterCrop(nn.Module):
    def __init__(self, crop_size: Union[int, Tuple[int, int]]):
        super().__init__()
        if isinstance(crop_size, tuple):
            self.crop_h, self.crop_w = crop_size
        else:
            self.crop_h = self.crop_w = crop_size
        
    def forward(self, image: PIL.Image):
        # image is a PIL.Image, not a torch.Tensor, so we need to use PIL.Image methods
        img_w, img_h = image.size
        
        # compute left, top, right, bottom
        # left & top will be included in the crop, right & bottom will be excluded
        <YOUR CODE>
        
        assert left >= 0, left
        assert top >= 0, top
        assert right < img_w, (right, img_w)
        assert bottom < img_h, (bottom, img_h)
        
        return image.crop((left, top, right, bottom))

assert CenterCrop(224)(PIL.Image.new('RGB', (719, 960))).size == (224, 224)
assert CenterCrop((224, 256))(PIL.Image.new('RGB', (719, 960))).size == (256, 224)

CenterCrop(224)(dataset_valid[0][0])

Ещё сделаем простейший класс-обёртку наподобие `Sequential`:

In [ ]:
from typing import List

class Compose(nn.Module):
    def __init__(self, submodules: List[nn.Module]):
        super().__init__()
        self.submodules = nn.ModuleList(submodules)

    def forward(self, image):
        <YOUR CODE>

Наконец, посмотрим на всю конструкцию в действии:

In [ ]:
dataset_valid = HotDogOrNotDataset(
    data_root, transform=Compose([
        Resize(256),
        CenterCrop(224),
    ]),
    split='test',
)
dataset_valid[0][0]

На этом мы заканчиваем ручную реализацию трансформаций и переходим на стандартные трансформации из `torchvision`.

In [ ]:
from torchvision import transforms

dataset_valid = HotDogOrNotDataset(
    data_root, transform=transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
    ]),
    split='test',
)
dataset_valid[0][0]

Для валидационной выборки, помимо `Resize`, `CenterCrop` и `Compose`, нам понадобятся ещё `ToTensor` и `Normalize`. `ToTensor` конвертирует `PIL.Image` в `torch.Tensor` и приблизительно эквивалентна следующему коду:

```python
def to_tensor(image):
    return torch.tensor(np.array(img) / 255.).permute((2, 0, 1))
```

`Normalize` вычитает из изображения фиксированный `mean` и делит на фиксированный `std`. Он нужен из-за того, что ResNet-18, который мы будем использовать, был обучен с применением такой нормализации.

In [ ]:
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

transform_valid = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

dataset_valid = HotDogOrNotDataset(data_root, transform=transform_valid, split='test')

dataset_valid[0][0]

## 1.5. Преобразования обучающего датасета

В обучении мы будем использовать простейшие аугментации:

* `RandomResizedCrop`: вырезать случайный прямоугольник из изображения, после чего привести его к фиксированному размеру;
* `RandomHorizontalFlip`: с вероятностью 0.5 отразить изображение по горизонтали.

Давайте посмотрим на эффект от этих аугментаций:

In [ ]:
dataset_train = HotDogOrNotDataset(
    data_root, transform=transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
    ]),
    split='train',
)
X, y = dataset_train[0]
print(dataset_train.classes[y])
X

Вернём `ToTensor` и `Normalize`:

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

dataset_train = HotDogOrNotDataset(data_root, transform=transform_train, split='train')

dataset_train[0][0]

## 1.6. Даталоадеры

Как обычно, заведём даталоадеры. Здесь я задал валидационному даталоадеру `shuffle=True`, чтобы потом была возможность посмотреть на случайный сэмпл предсказаний модели.

In [ ]:
batch_size = 64
num_workers = 2

dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, num_workers=num_workers, shuffle=True)
dataloader_valid = torch.utils.data.DataLoader(dataset_valid, batch_size=batch_size, num_workers=num_workers, shuffle=True)

# 2. Реквизируем ResNet-18 



In [ ]:
!nvidia-smi

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

## 2.1. Скачиваем предобученную модель

In [ ]:
model = torchvision.models.resnet18(pretrained=True).to(device)

# Не забываем про .eval(), чтобы отключить батчнормы
model.eval();

## 2.2. Смотрим, на что она способна

Теперь попробуем что-нибудь спрогнозировать. Вспомним, как мы делали это с VGG-16 на самой первой неделе.

In [ ]:
def get_image(url):
    response = requests.get(url)
    img = PIL.Image.open(io.BytesIO(response.content))
    return img

In [ ]:
IMG_URL = 'https://upload.wikimedia.org/wikipedia/en/5/5f/Original_Doge_meme.jpg'
# IMG_URL = 'https://sadanduseless.b-cdn.net/wp-content/uploads/2019/06/cat-breading4.jpg'
# IMG_URL = 'https://images-na.ssl-images-amazon.com/images/I/91NKh-FPcBL._SL1500_.jpg'
# IMG_URL = 'https://sun9-34.userapi.com/c850216/v850216669/110118/s1XSv_XLgtY.jpg'

image = get_image(IMG_URL)
print(f'Image size: {image.size}')
image 

In [ ]:
LABELS_URL = 'https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt'

response = requests.get(LABELS_URL)
labels = np.array(response.content.decode('utf-8').split('\n'))

print(f'Total labels: {len(labels)}')
print(f'Example labels: {labels[200:205]}')

Завернём весь процесс предсказания в функцию. Заодно познакомимся с функциональным интерфейсом к `torchvision.transforms`, чтобы можно было видеть, что именно получает на вход нейросеть:

In [ ]:
from IPython.display import display
import torchvision.transforms.functional

def predict(image):
    image = transforms.functional.resize(image, 256)
    image = transforms.functional.center_crop(image, 224)
    
    # Показываем картинку после resize и crop, но до преобразования в тензор:
    display(image)
    
    tensor = transforms.functional.to_tensor(image)
    tensor = transforms.functional.normalize(tensor, mean=IMAGENET_MEAN, std=IMAGENET_STD)
    
    # Добавьте размерность батча
    tensor = <YOUR CODE>
    # Перенесите tensor на GPU
    tensor = <YOUR CODE>
    
    # Предскажите logits_tensor, предварительно отключив сохранение вычислительного графа:
    <YOUR CODE>
    
    # Уберите размерность батча
    logits_tensor = <YOUR CODE>

    # Здесь можно было бы сразу посчитать и вернуть argmax,
    # если бы нас интересовало только одно предсказание.
    # Но мы посчитаем top-5 предсказаний. В прошлый раз мы
    # это делали таким кодом на Numpy:
    # 
    # logits = logits.cpu().numpy()
    # indices = logits.argsort()[-5:][::-1]
    # probs = scipy.special.softmax(logits)
    # 
    # Сейчас мы воспользуемся уже известной нам torch.nn.functional.softmax(),
    # а также полезной функцией torch.topk(), имеющей такой интерфейс:
    # 
    # >>> torch.topk(x, 5)
    # torch.return_types.topk(values=tensor([13.2613, 12.7950, 12.5249, 12.4262, 11.8144]), indices=tensor([260, 259, 273, 263, 151]))

    probs = <YOUR CODE>
    topk = <YOUR CODE>
    
    for idx in topk.indices:
        print(f'{probs[idx] * 100:>5.2f}% | {labels[idx]}')

In [ ]:
predict(image)

Напоследок посмотрим, как выглядят прогнозы на нашем датасете для хот-догов:

In [ ]:
X_batch, y_batch = next(iter(dataloader_valid))
X_batch = X_batch.to(device)
y_batch = y_batch.to(device)

with torch.no_grad():
    logits_tensor = model(X_batch)

probs_tensor = F.softmax(logits_tensor, dim=-1)
probs = probs_tensor.detach().cpu().numpy()

In [ ]:
cols = 5
rows = 3
fig, axarr = plt.subplots(rows, cols, figsize=(4 * cols - 1, 5 * rows - 1))

k = 0 
for i in range(rows):
    for j in range(cols):
        ax = axarr[i, j]
        ax.grid('off')
        ax.axis('off')
        ax.imshow(np.clip(X_batch[k].permute(1, 2, 0).cpu().numpy() * IMAGENET_STD + IMAGENET_MEAN, 0, 1))
        class_idx = probs[k].argmax()
        ax.set_title(f'{probs[k, class_idx]:>6.2%} : {labels[class_idx]}', size=14)
        k += 1
plt.tight_layout()
plt.show()

Модель отрабатывает на уровне выше всех похвал (но это неточно).

# 3. Хирургическое вмешательство

Побаловавшись с прогнозами, займёмся более серьёзными проблемами.

Предобученная сетка не приспособлена для работы с нашими классами. Давайте заставим её их выучить. Для этого нам придётся срезать с сетки её последние слои. Посмотрим на модель повнимательнее.

In [ ]:
model

In [ ]:
from torchsummary import summary

def print_summary(model):
    summary(model, (3, 224, 224), device=torch.device(device).type)

print_summary(model)

## 3.1. Вырезаем feature extractor

В прошлый раз, реализовывая эту модель, мы увидели, что она по сути является одной длинной последовательностью блоков, хотя почему-то в `torchvision` она не реализована как `nn.Sequential`.

Наша задача — переиспользовать как можно больше слоёв из этой модели. В данном случае мы можем переиспользовать вообще всё, кроме последнего линейного слоя, по сути делающего логистическую регрессию поверх свёрточных фичей.

Давайте создадим новую модель, которая будет возвращать эти самые свёрточные фичи. Для этого мы можем просто влезть внутрь предобученной модели, вытащить оттуда слои, и сделать из них новый `Sequential`. Единственный нюанс — после последнего усреднения получается тензор с шейпом `(B, 512, 1, 1)`, поэтому в конце надо сделать либо `Flatten`, либо несколько `squeeze()`.

In [ ]:
# Пример того, как можно влезть внутрь модели:
model.conv1

In [ ]:
# Посмотрите внимательно на напечатанную выше структуру модели,
# найдите все слои, которые там используются, и составьте из них
# один nn.Sequential. В нём должно получиться примерно 10 слоёв.

model_beheaded = <YOUR CODE>

assert model_beheaded(X_batch).shape == (batch_size, 512)

print_summary(model_beheaded)

## 3.2. Делаем feature extractor необучаемым

Дальше мы приделаем к этой части новый линейный слой, который обучим делать логистическую регрессию на новом датасете. Но сейчас нам нужно сделать так, чтобы эта часть не обучалась. Для этого нужно отключить опцию `requires_grad` у всех параметров внутри неё.

In [ ]:
def print_params_requires_grad(model):
    for name, p in model.named_parameters():
        print(f'{name:<30} {str(p.shape):<30} {p.requires_grad}')

print_params_requires_grad(model_beheaded)

In [ ]:
model_beheaded(X_batch).requires_grad

Делается это очень просто:

In [ ]:
for p in model_beheaded.parameters():
    p.requires_grad = False

In [ ]:
print_params_requires_grad(model_beheaded)

In [ ]:
model_beheaded(X_batch).requires_grad

## 3.3. Создаём новый классификатор поверх старого feature extractor

Теперь можно собрать новую модель.

In [ ]:
new_classifier = <YOUR CODE>
# Не забудьте перенести новую голову на GPU!

model_hotdog = <YOUR CODE>

print_summary(model_hotdog)

In [ ]:
print_params_requires_grad(model_hotdog)

In [ ]:
y_pred = model_hotdog(X_batch)
y_pred.shape

In [ ]:
y_pred.requires_grad

# 4. Tensorboard

Прежде чем это учить, познакомимся ещё с одним очень важным инструментом, полезным, когда вы ставите много долгих экспериментов, — Tensorboard. Это такая модная штука для визуализации логов.

Работает она так. Перед началом обучения вы создаёте объект класса `SummaryWriter`, который будет писать логи в специальную папку. Параллельно вы запустите процесс `tensorboard`, который будет читать эту папку и визуализировать в веб-интерфейсе то, что там найдёт.

По-хорошему, Tensorboard запускается из командной строки командой наподобие

```bash
tensorboard --port 6006 --logdir tb_logs
```

Но поскольку мы работаем в Colab, для нас всё будет устроено несколько иначе. Полную документацию можно посмотреть [тут](https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/tensorboard_in_notebooks.ipynb), но если вкратце, то, во-первых, нужно подгрузить Jupyter extension:

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

А потом запустить Tensorboard при помощи magic-команды, передав ей `--logdir`:

In [ ]:
%tensorboard --logdir tb_logs

Использовать `SummaryWriter` можно примерно так:

In [ ]:
# На случай перезапуска следующей ячейки удалим записанные логи
if Path('tb_logs/demo').exists():
    !rm -r tb_logs/demo

In [ ]:
from torch.utils.tensorboard import SummaryWriter

with SummaryWriter(log_dir='tb_logs/demo') as writer:
    for t in range(100):
        writer.add_scalar('some_tag', np.sin(t / 20), t)

# 5. Дообучение

А теперь давайте напишем функцию для обучения модели, но логи будем писать в Tensorboard:

In [ ]:
def train(model, criterion, opt, dataloader_train, dataloader_valid, num_epochs, run_name, device='cuda:0'):
    with SummaryWriter(log_dir=str(Path('tb_logs') / run_name)) as writer:
        train_batches = 0
        with tqdm(range(1, num_epochs + 1)) as epochs_progress_bar:
            for epoch in epochs_progress_bar:
                # Трейн
                model.train()
                with tqdm(dataloader_train, desc=f'Train | Epoch {epoch}') as train_progress_bar:
                    for x_batch, y_batch in train_progress_bar:
                        # Переносим батч на GPU
                        x_batch = x_batch.to(device)
                        y_batch = y_batch.to(device)

                        y_pred = model(x_batch)  # делаем предсказания
                        loss = criterion(y_pred, y_batch)  # считаем лосс

                        loss_val = loss.item()
                        writer.add_scalar('train/loss', loss_val, train_batches)
                        assert np.isfinite(loss_val)

                        # Считаем градиенты и делаем шаг оптимизатора, не забыв обнулить градиенты
                        opt.zero_grad()
                        loss.backward()
                        opt.step()

                        train_batches += 1

                model.eval()
                with torch.no_grad():
                    epoch_losses_valid = []
                    epoch_correct_predictions_valid = []
                    with tqdm(dataloader_valid, desc=f'Valid | Epoch {epoch}') as valid_progress_bar:
                        for x_batch, y_batch in valid_progress_bar:
                            # Переносим батч на GPU
                            x_batch = x_batch.to(device)
                            y_batch = y_batch.to(device)

                            y_pred = model(x_batch)  # делаем предсказания
                            loss = criterion(y_pred, y_batch)  # считаем лосс
                            
                            loss_val = loss.item()
                            assert np.isfinite(loss_val)
                            epoch_losses_valid.append(loss_val)

                            batch_correct_predictions = torch.argmax(y_pred, dim=-1) == y_batch
                            epoch_correct_predictions_valid.extend(batch_correct_predictions.to('cpu').numpy().tolist())

                    writer.add_scalar('valid/loss', np.mean(epoch_losses_valid), epoch)
                    writer.add_scalar('valid/accuracy', np.mean(epoch_correct_predictions_valid), epoch)

Как обычно, создадим `criterion`, `opt`...

In [ ]:
criterion = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model_hotdog.parameters(), lr=1e-3)

Полезный костыль: при записи логов Tensorboard в названии папки указывать текущее время. Код для этого:

In [ ]:
import datetime

def get_datetime():
    return datetime.datetime.now().isoformat(sep='_', timespec='milliseconds').replace(':', '-')

get_datetime()

Запускаем дообучение!

In [ ]:
train(
    model_hotdog, criterion, opt, dataloader_train, dataloader_valid,
    num_epochs=5, run_name=f'{get_datetime()}_finetune-resnet18-5-epochs', device=device)

# 5. Смотрим на результаты

In [ ]:
X_batch, y_batch = next(iter(dataloader_valid))
X_batch = X_batch.to(device)
y_batch = y_batch.to(device)

with torch.no_grad():
    logits_tensor = model_hotdog(X_batch)

probs_tensor = F.softmax(logits_tensor, dim=-1)
probs = probs_tensor.detach().cpu().numpy()

In [ ]:
[dataset_valid.classes[idx] for idx in probs.argmax(axis=1)]

In [ ]:
cols = 5
rows = 3
fig, axarr = plt.subplots(rows, cols, figsize=(4 * cols - 1, 5 * rows - 1))

k = 0 
for i in range(rows):
    for j in range(cols):
        ax = axarr[i, j]
        ax.grid('off')
        ax.axis('off')
        ax.imshow(np.clip(X_batch[k].permute(1, 2, 0).cpu().numpy() * IMAGENET_STD + IMAGENET_MEAN, 0, 1))
        class_idx = probs[k].argmax()
        ax.set_title(f'{probs[k, class_idx]:>6.2%} : {dataset_valid.classes[class_idx]}', size=14)
        k += 1
plt.tight_layout()
plt.show()